# Sloping Beach test case

The DG-CG code suffers from oscillations which appear to follow the orientation of the mesh. We will plot the cross-section elevation over time and see where the oscillation originates.

The current configuration is 7 cells wide, with numbering as follows:


| | | | | | |
|---|---|---|---|---|---|
| 2 | 72 | 142 | 212 | ... | 492 |
| 1  | 71 | 141 | 211 | ... | 491 |

In other words, the increment is 70 along a row, and 1 along a column.

To generate this mesh, run

    python slopingbeach.py 10000 fort.14

and then rename `fort.14.15` to `fort.15`.

To run the various configurations (ADCIRC, ADCIRC-DG, etc.), go into the `Beach` directory and use the makefile.

## Extracting data from fort.63

For this test case, we will only look at the x- and y-direction nodes. If looking at x-profile, specify the `row` variable with a value from 1 to 70. If looking at the y-profile, the `col` variable can be 1, 71, ..., 491.

We will use the function `read_63()` to extract the time series data of those nodes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from utils_beach import *
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline
%load_ext autoreload
%autoreload 2


### Case 1: DG-CG creating spurious oscillations

This case compares ADCIRC with ADCIRC-DG. To generate the fort.63 files, run

    make adc-serial
    make coupling-serial

and load the data using the cell above.

In [ ]:
row = 2
col = 211
NP = 560 # no. of nodes

N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]
y_adg = read_63('./Beach/fort.63.coupling', y_nodes, total_nodes=NP)
y_adc = read_63('./Beach/fort.63.adc', y_nodes, total_nodes=NP)

v_adg = read_63('./Beach/fort.64.coupling', y_nodes, total_nodes=NP, col=2)
v_adc = read_63('./Beach/fort.64.adc', y_nodes, total_nodes=NP, col=2)
data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }

data2 = {
    "ADCIRC": v_adc,
    "DG-CG": v_adg
}
data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
HTML(ani.to_jshtml())

#### Cross-section view

In [ ]:
row = 2
col = 211
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
y_nodes = [row + M*i for i in range(0,N)]
y_adg = read_63('./Beach/fort.63.coupling', y_nodes, total_nodes=NP)
y_adc = read_63('./Beach/fort.63.adc', y_nodes, total_nodes=NP)

v_adg = read_63('./Beach/fort.64.coupling', y_nodes, total_nodes=NP, col=1)
v_adc = read_63('./Beach/fort.64.adc', y_nodes, total_nodes=NP, col=1)
data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }

data2 = {
    "ADCIRC": v_adc,
    "DG-CG": v_adg
}
data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'x-velocity'], title='x-direction profile')
HTML(ani.to_jshtml())

### Case 2: isolated DG elevation

In this case, we run the program using ADCIRC. The DG solver then solves for elevation alongside it using velocities from `solveMomentumEq()`, but does not feed it back to it in the next timestep. This case is done to investigate if DG produces incorrect elevation despite using the correct velocities. Instead, the DG elevation values are stored in `ETA3`.

To run, use 

    make eta3

which then calls the program with `--solver 10`. This option prints out values of `ETA3` instead of `ETA2` to the fort.63 file.

In [ ]:
row = 2
col = 211
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction

cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
y_eta3 = read_63('Beach/fort.63.eta3', y_nodes, total_nodes=NP)
v_eta3 = read_63('Beach/fort.64.eta3', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG isolated elevation": y_eta3
       }
data2 = {
    "ADCIRC" : v_eta3
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 3: Constant bathymetry

Here we used the command

    python --bathy -200 10000 fort.14

to generate the mesh with a constant bathymetry 200m below geoid.

**Update**: Now accept a command line argument `--bathy` which overrides the bathymetry in the fort.14 with the supplied value. Run

    make const BATHY=-200

#### 3a: Deep bathymetry (200m below geoid)

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes

N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.coupling.-200', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc.-200', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.coupling.-200', y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc.-200', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

#### 3b: Shallow bathymetry (10 m below geoid)

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes

N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.coupling.-10', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc.-10', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.coupling.-10', y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc.-10', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG": v_adg
}

data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
HTML(ani.to_jshtml())

#### 3c: P0 DG (10m below geoid)
Here we use a constant cell-average for DG (finite volume) and use a 10m bathymetry. Run with

    make p0 BATHY=-10

which uses the option `--solver 2`.

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes

N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.p0', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc.-200', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.p0', y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc.-200', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG (p = 0)": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG (p = 0)": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 4: 'X' mesh

This mesh was created with `trirectangle.m` through the `beach_meshes.m` script. The triangulation here is

         ____          ____ 
        |    |        |\  /|
        |    |  ==>   | \/ | 
        |    |        | /\ |
        |____|        |/__\|


In [ ]:
col = 1
NP = 1182 # no. of nodes
M = 70 # no. of nodes in the y-direction
y_nodes = [col + i for i in range(M)]

y_adg = read_63('BeachX/fort.63.coupling',y_nodes, total_nodes=NP)
y_adc = read_63('BeachX/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('BeachX/fort.64.coupling',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('BeachX/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 5: 'X' mesh with isolated DG elevation

In [ ]:
col = 1
NP = 1182 # no. of nodes
M = 70 # no. of nodes in the y-direction
y_nodes = [col + i for i in range(M)]

y_adg = read_63('BeachX/fort.63.eta3',y_nodes, total_nodes=NP)
y_adc = read_63('BeachX/fort.63.adc', y_nodes, total_nodes=NP)
v_adc = read_63('BeachX/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG isolated elevation": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 6: re-project nodal-averaged elevation back to DG

Here we project the nodal-averaged `eta2` back to the DG basis and compute the next timestep with it. The rationale here is to "sync up" the solution between elevation and momentum.

Run with 

    make reproject

which calls the code with option `--solver 3`

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.reproj', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.reproj',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 7: DG elevation in H terms, GWCE in gradient terms

Ethan's suggestion: feed the CG momentum solver with DG nodal-averaged elevation only for non-gradient terms, i.e.

$$ \left\langle \frac{\tau}{H \rho_0}, \phi_j \right\rangle_\Omega = \left\langle \frac{\tau}{(\zeta + h) \rho_0}, \phi_j \right\rangle_\Omega$$

but use the GWCE elevation to compute the gradient term in the **barotropic** pressure:

$$ \left \langle g \frac{\partial \zeta}{\partial x}, \phi_j \right\rangle_\Omega $$

These are stored in the variables `DBTPDX1A`, etc. in `momentum.F`.

This can be run with

    make gradient

which calls the program with `--solver 4`.

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.grad', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.grad',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adg)/len(y_nodes))-1, step=1, value=0))

### Case 8: DG + Conservative momentum formulation

Use

    make cm

which calls the options `--solver 1 --momentum_solver 2`

In [ ]:
row = 2
col = 1
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.cm', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc.-200', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.cm',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc.-200', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG, conservative momentum": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG, conservative momentum": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adc)/len(y_nodes))-1, step=1, value=0))

### Case 9 : New momentum formulation


In [ ]:
row = 2
col = 71
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.new', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc.-10', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.new',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc.-10', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG, new momentum": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG, new momentum": v_adg
}

data_list = [data1, data2]
f = make_plot_function(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
#interact(f, frame=widgets.IntSlider(min=0, max=int(len(y_adg)/len(y_nodes))-1, step=1, value=0))
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
HTML(ani.to_jshtml())

### Case 10: Using DG cell average
In this case, the nodal average of each `ETA2(I)` is computed using the cell average of each neighboring cell, while the DG solver still maintains a first-order solution state in `ZE`. Run with

    make avg

which uses the option `--solver 5`.

In [ ]:
row = 2
col = 71
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.avg', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.avg',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG, cell-average": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG, cell-average": v_adg
}

data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
HTML(ani.to_jshtml())

#### Cross-section view

In [ ]:
row = 2
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
y_nodes = [row + M*i for i in range(0,N)]

y_adg = read_63('Beach/fort.63.avg', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.avg',y_nodes, total_nodes=NP, col=1)
v_adc = read_63('Beach/fort.64.adc', y_nodes, total_nodes=NP, col=1)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG, cell-average": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG, cell-average": v_adg
}

data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'x-velocity'], title='x-direction profile')
HTML(ani.to_jshtml())

### Case 11: Stabilization term


In [ ]:
row = 2
col = 71
NP = 560 # no. of nodes
N = 8 # no. of nodes in the x-direction
M = 70 # no. of nodes in the y-direction
cross_nodes = [row + M*i for i in range(0,N)]
y_nodes = [col + i for i in range(M)]

y_adg = read_63('Beach/fort.63.stab', y_nodes, total_nodes=NP)
y_adc = read_63('Beach/fort.63.adc', y_nodes, total_nodes=NP)
v_adg = read_63('Beach/fort.64.stab',y_nodes, total_nodes=NP, col=2)
v_adc = read_63('Beach/fort.64.adc', y_nodes, total_nodes=NP, col=2)

data1 = {
        "ADCIRC": y_adc,
        "DG-CG, cell-average": y_adg
       }
data2 = {
    "ADCIRC" : v_adc,
    "DG-CG, cell-average": v_adg
}

data_list = [data1, data2]
ani = animate63(data_list, node_list=y_nodes, ylabel=['Water elevation', 'y-velocity'])
HTML(ani.to_jshtml())